In [1]:
import os
import re
import sys
import warnings
from typing import List, Tuple

import camelot
import pandas as pd
import pdf2image
import philaudit as pa
import PyPDF2 as p
from PyPDF2.errors import PdfReadError
from tqdm import tqdm

PYDEVD_WARN_EVALUATION_TIMEOUT = 10.0

In [2]:
pdf='./new_testers/01-Flora2019_Audit_Report.pdf'
reader = p.PdfReader(pdf)

In [3]:
reader = p.PdfReader(pdf)
try:
    start, end = pa.assign_start_end_pages(reader)
    part3_rng = pa.detect_range_from_target_page(pdf, reader, start, end)
    part3_rng = pa.remove_high_image_area_pages(pdf, part3_rng)
    part3_rng = pa.remove_annex_pages_from_end_of_range(reader, part3_rng)
    part3_rng = pa.remove_cover_pages_from_range(reader, part3_rng)
    page_ranges = pa.remove_blank_pages_from_range(pdf, reader, part3_rng)
    page_ranges = pa.remove_pages_without_tables(pdf, page_ranges) # <-- Would love a faster way to do this... 
    page_ranges
except Exception as e:
    print(e)

In [ ]:
table_pages = []
for rng in page_ranges:
    table_pages.extend(rng)
table_pages

In [ ]:
run_data = [(pdf, table_pages)]

In [ ]:
df = pd.DataFrame(run_data, columns=["file", "part3_range"])
df["pg_count"] = df["part3_range"].apply(lambda x: len(x))
df

In [ ]:
!rm -r ./test_pipeline_images/*

In [ ]:
output_dir = "./test_pipeline_images/"
for _, row in df.iterrows():
    # convert_pages_to_bitmap(row.file, output_dir="./test_pipeline_images/", part3_ranges = row.part3_range)
    pgs = row.part3_range
    images = pdf2image.convert_from_path(pdf)
    for pg_num in pgs:
        file = row.file.split("/")[-1]
        images[pg_num].save(
            os.path.join(output_dir,
                         f"{file}_page_{pg_num + 1}.bmp"), "BMP")
    

In [ ]:
images = pdf2image.convert_from_path(pdf)
images[0]

In [ ]:
images[-1]